# Отток клиентов

Из банка стали уходить клиенты. С целью фокуса маркетинговой стратегии нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Имеются исторические данные о поведении клиентов и расторжении договоров с банком. 

Построи модель с предельно большим значением *F1*-меры (не менее 0.59).

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Подготовка данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/Churn.csv')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None


Типы данных соответствуют. Выявлены пропуски по столбцу Tenure (количество недвижимости). проанализируем детально

In [2]:
print(data['Tenure'].unique())
print(data['Tenure'].value_counts())

[ 2.  1.  8.  7.  4.  6.  3. 10.  5.  9.  0. nan]
1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64


Причина пропусков - при заполнении данных респонденты не заполнили поле, т.к. у них не было объектов недвижимости во владении. Заполним пропуски 0.

In [3]:
data = data.fillna(value=0)
print(data['Tenure'].value_counts())

0.0     1291
1.0      952
2.0      950
8.0      933
3.0      928
5.0      927
7.0      925
4.0      885
9.0      882
6.0      881
10.0     446
Name: Tenure, dtype: int64


Первые три столбца - индекс строки в данных, уникальный идентификатор клиента, фамилия - не являются признаками, на которых можно строить предсказания. Откинем эти столбцы.

In [4]:
churn = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
print(churn.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB
None


Преобразуем категориальные переменные в численные с помощью метода прямого кодирования. Чтобы избежать дамми-ловушки, удалим первый столбцец, значение которого можно восстановить по другим.

In [5]:
churn_ohe = pd.get_dummies(churn, drop_first=True)
print(churn_ohe.shape)
print(churn_ohe.head())

(10000, 12)
   CreditScore  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          619   42     2.0       0.00              1          1   
1          608   41     1.0   83807.86              1          0   
2          502   42     8.0  159660.80              3          1   
3          699   39     1.0       0.00              2          0   
4          850   43     2.0  125510.82              1          1   

   IsActiveMember  EstimatedSalary  Exited  Geography_Germany  \
0               1        101348.88       1                  0   
1               1        112542.58       0                  0   
2               0        113931.57       1                  0   
3               0         93826.63       0                  0   
4               1         79084.10       0                  0   

   Geography_Spain  Gender_Male  
0                0            0  
1                1            0  
2                0            0  
3                0            0  
4                1

Разделим полученный датасет на 3 выборки.

In [6]:
from sklearn.model_selection import train_test_split
churn_train, churn_valid_test = train_test_split(churn_ohe, test_size=0.4, random_state=12345)
churn_valid, churn_test = train_test_split(churn_valid_test, test_size=0.5, random_state=12345)

Целевой признак для нас - это столбец Exited - ушел клиент или нет. Разделим полученные выборки на целевые признаки и признаки. 

In [7]:
train_features = churn_train.drop(['Exited'], axis=1)
train_target = churn_train['Exited']
valid_features = churn_valid.drop(['Exited'], axis=1)
valid_target = churn_valid['Exited']
test_features = churn_test.drop(['Exited'], axis=1)
test_target = churn_test['Exited']

Т.к. в наших данных присутствуют колличественные признаки с разными разбросами значений, то алгорим может решить, что признаки с
большими значениями и разбросом важнее. Чтобы избежать этой ловушки, отмасшатибруем признаки (приведен к одному масштабу).

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#Обучим scaler на обучающей выборке
scaler.fit(train_features)

#отмасштабируем тренировочные, валидационные и тестовые выборки
train_features_scaled = scaler.transform(train_features)
valid_features_scaled = scaler.transform(valid_features)
test_features_scaled = scaler.transform(test_features)

Вывод

Мы ознакомились с данными, заполнили пропуски по столбцу Tenure, перекодировали ктегориальные переменные, разделили данные на 3 выборки, отмасштабировали данные.

# 2. Исследование задачи

Изучим баланс классов. Классы являются сбалансированными, когда их соотношение 1:1. 

In [9]:
print(data['Exited'].value_counts())

0    7963
1    2037
Name: Exited, dtype: int64


Мы видим, что наблюдается дисбаланс классов (1 - 20%, 0 - 80%). Соответственно, метрика accruacy не является подходящей. Для таких случаев есть метрика полнота (recall) и точность (precision). Комибированной метрикой из этих двух является F1. По ней и будем оценивать качество моделей.

Выберем наиболее подходящую модель из случайного леса и логистической регрессии. Начнем со случайного леса. Переберем гиперпараметры и выберем модель, которая даёт наибольшую метрику f1.

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

f1_best = 0
estim_best = 0
depth_best = 0

for estim in range (10, 101, 10):
    for depth in range(1, 11):
        forest = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=12345)
        forest.fit(train_features_scaled, train_target)
        prediction = forest.predict(valid_features_scaled)
        f1 = f1_score(valid_target, prediction)
        if f1>f1_best:
            f1_best=f1
            estim_best=estim
            depth_best=depth
        print('F1-score при значении n_estimators =', estim, 'и max_depth = ', depth, 'составила:', f1)

print('Лучший F1-score при значении n_estimators =', estim_best, 'и max_depth = ', depth_best, 'составила:', f1_best)

F1-score при значении n_estimators = 10 и max_depth =  1 составила: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


F1-score при значении n_estimators = 10 и max_depth =  2 составила: 0.2
F1-score при значении n_estimators = 10 и max_depth =  3 составила: 0.25102880658436216
F1-score при значении n_estimators = 10 и max_depth =  4 составила: 0.476843910806175
F1-score при значении n_estimators = 10 и max_depth =  5 составила: 0.4727891156462586
F1-score при значении n_estimators = 10 и max_depth =  6 составила: 0.5463258785942492
F1-score при значении n_estimators = 10 и max_depth =  7 составила: 0.5408805031446541
F1-score при значении n_estimators = 10 и max_depth =  8 составила: 0.5527156549520766
F1-score при значении n_estimators = 10 и max_depth =  9 составила: 0.5800604229607251
F1-score при значении n_estimators = 10 и max_depth =  10 составила: 0.5735963581183612
F1-score при значении n_estimators = 20 и max_depth =  1 составила: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


F1-score при значении n_estimators = 20 и max_depth =  2 составила: 0.15789473684210525
F1-score при значении n_estimators = 20 и max_depth =  3 составила: 0.2401656314699793
F1-score при значении n_estimators = 20 и max_depth =  4 составила: 0.41379310344827586
F1-score при значении n_estimators = 20 и max_depth =  5 составила: 0.5042301184433163
F1-score при значении n_estimators = 20 и max_depth =  6 составила: 0.5361842105263157
F1-score при значении n_estimators = 20 и max_depth =  7 составила: 0.5411392405063291
F1-score при значении n_estimators = 20 и max_depth =  8 составила: 0.5632911392405064
F1-score при значении n_estimators = 20 и max_depth =  9 составила: 0.5749613601236476
F1-score при значении n_estimators = 20 и max_depth =  10 составила: 0.5787878787878789
F1-score при значении n_estimators = 30 и max_depth =  1 составила: 0.0


...

F1-score при значении n_estimators = 80 и max_depth =  1 составила: 0.0
F1-score при значении n_estimators = 80 и max_depth =  2 составила: 0.12975391498881433
F1-score при значении n_estimators = 80 и max_depth =  3 составила: 0.23060796645702306
F1-score при значении n_estimators = 80 и max_depth =  4 составила: 0.38130841121495324
F1-score при значении n_estimators = 80 и max_depth =  5 составила: 0.48453608247422686
F1-score при значении n_estimators = 80 и max_depth =  6 составила: 0.5385878489326765
F1-score при значении n_estimators = 80 и max_depth =  7 составила: 0.5530546623794212
F1-score при значении n_estimators = 80 и max_depth =  8 составила: 0.5645933014354068
F1-score при значении n_estimators = 80 и max_depth =  9 составила: 0.5628930817610063
F1-score при значении n_estimators = 80 и max_depth =  10 составила: 0.5793528505392913


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


F1-score при значении n_estimators = 90 и max_depth =  1 составила: 0.0
F1-score при значении n_estimators = 90 и max_depth =  2 составила: 0.11286681715575622
F1-score при значении n_estimators = 90 и max_depth =  3 составила: 0.22315789473684208
F1-score при значении n_estimators = 90 и max_depth =  4 составила: 0.3714821763602251
F1-score при значении n_estimators = 90 и max_depth =  5 составила: 0.46956521739130436
F1-score при значении n_estimators = 90 и max_depth =  6 составила: 0.5320197044334976
F1-score при значении n_estimators = 90 и max_depth =  7 составила: 0.5530546623794212
F1-score при значении n_estimators = 90 и max_depth =  8 составила: 0.5591054313099042
F1-score при значении n_estimators = 90 и max_depth =  9 составила: 0.5660377358490566
F1-score при значении n_estimators = 90 и max_depth =  10 составила: 0.5771604938271605


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


F1-score при значении n_estimators = 100 и max_depth =  1 составила: 0.0
F1-score при значении n_estimators = 100 и max_depth =  2 составила: 0.1294642857142857
F1-score при значении n_estimators = 100 и max_depth =  3 составила: 0.23749999999999996
F1-score при значении n_estimators = 100 и max_depth =  4 составила: 0.37453183520599254
F1-score при значении n_estimators = 100 и max_depth =  5 составила: 0.46234676007005243
F1-score при значении n_estimators = 100 и max_depth =  6 составила: 0.5370675453047776
F1-score при значении n_estimators = 100 и max_depth =  7 составила: 0.5507246376811594
F1-score при значении n_estimators = 100 и max_depth =  8 составила: 0.5582137161084529
F1-score при значении n_estimators = 100 и max_depth =  9 составила: 0.5678233438485806
F1-score при значении n_estimators = 100 и max_depth =  10 составила: 0.5793528505392913
Лучший F1-score при значении n_estimators = 60 и max_depth =  10 составила: 0.5806451612903226


Лучший F1-score при значении n_estimators = 60 и max_depth =  10 составил 0.5806451612903226

In [11]:
from sklearn.linear_model import LogisticRegression
regression = LogisticRegression(random_state=12345)
regression.fit(train_features_scaled, train_target)
prediction = regression.predict(valid_features_scaled)
f1 = f1_score(valid_target, prediction)
print('F1-score составила:', f1)

F1-score составила: 0.3333333333333333


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Ни случайный лес, ни логистическая регрессия не достигают значения f1 > 0.59. ПРичина тому, что данные не сбалансированы. В следующем шаге повысим качество модели, устранив дисбаланс.

Вывод

Мы изучили баланс классов, посмотрели, какое значение метрики F1 дают различные модели (при различных гиперпараметрах). Пришли к выводу, что они не достигают требуемой показателя более 0.59.

# 3. Борьба с дисбалансом

Для борьбы с дисбалансом проанализируем 2 способа: взвешивание классов и увеличение/уменьшение выборки. Для проверки модели на вменяемость дополнительно к F1 будем рассчитывает AUC-ROC.  Начнем со взвешивания классов.

In [12]:
from sklearn.metrics import roc_auc_score

f1_best = 0
estim_best = 0
depth_best = 0
auc_roc_best = 0

for estim in range (10, 101, 10):
    for depth in range(1, 11):
        forest = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=12345, class_weight='balanced')
        forest.fit(train_features_scaled, train_target)
        prediction = forest.predict(valid_features_scaled)
        f1 = f1_score(valid_target, prediction)
        
        probabilities_valid = forest.predict_proba(valid_features_scaled)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(valid_target, probabilities_one_valid)
        
        if f1>f1_best:
            f1_best=f1
            estim_best=estim
            depth_best=depth
            auc_roc_best=auc_roc

print('')
print('Лучший F1-score = {:.4f}, AUC-ROC score = {:.4f} при значении n_estimators = {} и max_depth = {}'.format(f1_best, auc_roc_best, estim_best, depth_best))



Лучший F1-score = 0.6346, AUC-ROC score = 0.8571 при значении n_estimators = 70 и max_depth = 7


In [13]:
regression = LogisticRegression(random_state=12345, class_weight='balanced', solver='liblinear')
regression.fit(train_features_scaled, train_target)
prediction = regression.predict(valid_features_scaled)
f1 = f1_score(valid_target, prediction)

probabilities_valid = regression.predict_proba(valid_features_scaled)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(valid_target, probabilities_one_valid)


print('F1-score составила: {:.4f}'.format(f1))
print('AUC-ROC score составила: {:.4f}'.format(auc_roc))

F1-score составила: 0.4889
AUC-ROC score составила: 0.7634


Качество обеих моделей увеличилось, но логистическая регрессия не достигла требуемого показателя, а вот случайный лес смог его превысить. Попробуем улучшить качество модели за счет увеличения выборки. Напишем формулу для upsampling.

In [14]:
from sklearn.utils import shuffle

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(
    features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

Ранее мы установили, что 1 - это 20% от всех ответов. Поэтому увеличим их долю в 4 раза.

In [15]:
train_features_upsampled, train_target_upsampled = upsample(train_features, train_target, 4)

In [16]:
#отмасштабируем тренировочные тестовые выборки
train_features_upsampled_scaled = scaler.transform(train_features_upsampled)

Рассмотрим случайный лес

In [17]:
f1_best = 0
estim_best = 0
depth_best = 0
auc_roc_best = 0

for estim in range (10, 101, 10):
    for depth in range(1, 11):
        forest = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=12345)
        forest.fit(train_features_upsampled_scaled, train_target_upsampled)
        prediction = forest.predict(valid_features_scaled)
        f1 = f1_score(valid_target, prediction)
        
        probabilities_valid = forest.predict_proba(valid_features_scaled)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(valid_target, probabilities_one_valid)
        
        if f1>f1_best:
            f1_best=f1
            estim_best=estim
            depth_best=depth
            auc_roc_best=auc_roc

print('')
print('Лучший F1-score = {:.4f}, AUC-ROC score = {:.4f} при значении n_estimators = {} и max_depth = {}'.format(f1_best, auc_roc_best, estim_best, depth_best))



Лучший F1-score = 0.6255, AUC-ROC score = 0.8552 при значении n_estimators = 20 и max_depth = 7


Рассмотрим логистическую регрессию

In [18]:
regression = LogisticRegression(random_state=12345, solver='liblinear')
regression.fit(train_features_upsampled_scaled, train_target_upsampled)
prediction = regression.predict(valid_features_scaled)
f1 = f1_score(valid_target, prediction)

probabilities_valid = regression.predict_proba(valid_features_scaled)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(valid_target, probabilities_one_valid)


print('F1-score составила: {:.4f}'.format(f1))
print('AUC-ROC score составила: {:.4f}'.format(auc_roc))

F1-score составила: 0.4889
AUC-ROC score составила: 0.7634


Наилучшее качество модели было достигнуто при взвешивании классов с испльзование случайного леса, с глубиной 7 и количеством дерьевев 70. 

# 4. Тестирование модели

In [19]:
best_model = RandomForestClassifier(n_estimators=70, max_depth=7, random_state=12345, class_weight='balanced')
best_model.fit(train_features_scaled, train_target)
prediction = best_model.predict(test_features_scaled)
f1 = f1_score(test_target, prediction)
        
probabilities_valid = best_model.predict_proba(test_features_scaled)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(test_target, probabilities_one_valid)
print('F1-score = {:.4f}, AUC-ROC score = {:.4f}'.format(f1, auc_roc))



F1-score = 0.6098, AUC-ROC score = 0.8523


Достигнут целевой показатель F1, а также подтверждена вменяемость модель (AUC-ROC > 0.5.